In [2]:

import tensorflow as tf
import tensorflow_hub as hub
import bert

import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_csv("age.csv")

# Define the labels and corresponding texts
labels = df['age'].values
texts = df['Content'].values

# Tokenize the texts
tokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenizer(vocabulary_file, to_lower_case)

def encode_text(text, tokenizer, max_len=512):
    tokens = tokenizer.tokenize(text)
    tokens = tokens[:max_len-2]
    input_sequence = ["[CLS]"] + tokens + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_sequence)
    input_mask = [1] * len(input_ids)
    padding = [0] * (max_len - len(input_ids))
    input_ids += padding
    input_mask += padding
    segment_ids = [0] * max_len
    return input_ids, input_mask, segment_ids

encoded_texts = [encode_text(text, tokenizer) for text in texts]
encoded_texts = np.array(encoded_texts)
input_ids = encoded_texts[:, 0]
input_masks = encoded_texts[:, 1]
segment_ids = encoded_texts[:, 2]

# Convert the labels to one-hot encoding
labels = pd.get_dummies(labels).values

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(encoded_texts, labels, test_size=0.2, random_state=0)

# Build the model
input_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name="input_ids")
input_mask = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(512,), dtype=tf.int32, name="segment_ids")


2023-02-10 23:15:40.836974: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


In [4]:

bert_model = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2')

pooled_output, sequence_output = bert_layer([input_ids, input_mask, segment_ids])

x = tf.keras.layers.Dense(128, activation='relu')(pooled_output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)

output = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(inputs=[input_ids, input_mask, segment_ids], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2023-02-10 23:19:01.779691: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 15627264 exceeds 10% of free system memory.


In [5]:
input_ids = tokenizer.encode(texts, add_special_tokens=True)
input_ids_test = texts.tensor([input_ids])

# Generate the input mask
input_mask_test = texts.ones_like(input_ids_test)

# Generate the segment IDs
segment_ids_test = texts.zeros_like(input_ids_test)


AttributeError: 'FullTokenizer' object has no attribute 'encode'

In [6]:
# Train the model
history = model.fit([input_ids_train, input_mask_train, segment_ids_train], y_train, epochs=10, batch_size=32, validation_data=([input_ids_test, input_mask_test, segment_ids_test], y_test))


NameError: name 'input_ids_train' is not defined

In [ ]:

# Evaluate the model
scores = model.evaluate([input_ids_test, input_mask_test, segment_ids_test], y_test)
print("Accuracy:", scores[1])